In [17]:
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer,
    TextClassificationPipeline,
)
import torch

In [18]:
THRESHOLD = 0.5

In [19]:
MODEL_PATH = "CarusoVitor/multiemotion-portuguese"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
labels = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Neutral']
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [21]:
model = AutoModelForSequenceClassification.from_pretrained( 
    MODEL_PATH,
    problem_type="multi_label_classification",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [22]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, function_to_apply="sigmoid")


Device set to use cpu
/home/vitor/.cache/pypoetry/virtualenvs/pln-trabalho-final-9al6VtFs-py3.10/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
scores = pipe("meu deus que raiva eu não acredito nisso")[0]
predicted = [item["label"] for item in scores if item["score"] > THRESHOLD]
predicted

['Anger', 'Surprise']

In [27]:
scores

[{'label': 'Anger', 'score': 0.9919611811637878},
 {'label': 'Disgust', 'score': 0.07572012394666672},
 {'label': 'Fear', 'score': 0.03407902643084526},
 {'label': 'Joy', 'score': 0.02200358733534813},
 {'label': 'Sadness', 'score': 0.057511068880558014},
 {'label': 'Surprise', 'score': 0.5826078653335571},
 {'label': 'Neutral', 'score': 0.001974120968952775}]